In [ ]:
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from rasterio.features import geometry_mask
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Шаг 1: Загрузка растрового изображения
print("Начало загрузки растрового изображения...")
raster_path = 'Vasilevskiy.tif'
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # Чтение первого канала
    raster_transform = src.transform
    raster_crs = src.crs  # Система координат растра
print("Загрузка растрового изображения завершена.")

# Шаг 2: Загрузка векторных данных
print("Начало загрузки векторных данных...")
vector_path = 'GA.geojson'  # Замените на путь к вашему векторному файлу
gdf = gpd.read_file(vector_path)
print("Загрузка векторных данных завершена.")

# Преобразование геометрий к системе координат растра
print("Преобразование геометрий векторных данных к системе координат растра...")
gdf = gdf.to_crs(raster_crs)
print("Преобразование завершено.")

# Шаг 3: Настройка графика
print("Настройка графика...")
fig, ax = plt.subplots(figsize=(10, 10))

# Шаг 4: Отображение растрового изображения
print("Отображение растрового изображения...")
plt.imshow(raster_data, extent=(raster_transform[2], raster_transform[2] + raster_transform[0] * raster_data.shape[1],
                                  raster_transform[5] + raster_transform[4] * raster_data.shape[0], raster_transform[5]),
           cmap='gray', alpha=0.5)  # alpha для прозрачности

# Шаг 5: Наложение векторных данных
print("Наложение векторных данных на график...")
gdf.plot(ax=ax, facecolor='none', edgecolor='red')  # Отображение векторных данных

# Шаг 6: Настройка графика
plt.title('Overlay Raster on Vector')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid()
plt.show()

def extract_features(raster_data, gdf, raster_transform):
    print("Начало извлечения признаков...")
    features = []
    labels = []
    
    for _, row in gdf.iterrows():
        geometry = row['geometry']
        label = row['Label']  # Убедитесь, что этот столбец существует
        
        # Создание маски с использованием rasterio
        mask = geometry_mask(
            [geometry], 
            out_shape=raster_data.shape, 
            transform=raster_transform, 
            invert=True
        )
        
        pixel_values = raster_data[mask]
        
        if len(pixel_values) > 0:
            features.append(pixel_values.reshape(-1, 1))  # Изменяем форму на (n_samples, 1)
            labels.extend([label] * len(pixel_values))
        else:
            print(f"No pixels found for label {label}")

    print("Извлечение признаков завершено.")
    return (np.concatenate(features) if features else np.array([]), 
            np.array(labels))

# Извлечение признаков
X, y = extract_features(raster_data, gdf, raster_transform)

# Проверка на пустые массивы
if X.size == 0 or y.size == 0:
    raise ValueError("No data available for training. Please check your input data.")

# Шаг 5: Разделение данных на обучающую и тестовую выборки
print("Начало разделения данных на обучающую и тестовую выборки...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Разделение данных завершено.")

# Шаг 6: Обучение классификатора KNN
print("Начало обучения классификатора KNN...")
classifier = KNeighborsClassifier(n_neighbors=8, metric='manhattan')  # Используем KNN с 5 соседями
classifier.fit(X_train, y_train)
print("Обучение завершено.")

# Шаг 7: Классификация тестовых данных
print("Начало классификации тестовых данных...")
y_pred = classifier.predict(X_test)
print("Классификация завершена.")

# Шаг 8: Оценка модели
print("Оценка модели...")
print(classification_report(y_test, y_pred))

# Шаг 9: Применение классификатора к всему изображению
def classify_image(image, classifier):
    print("Начало классификации всего изображения...")
    height, width = image.shape[:2]
    classified_image = np.zeros((height, width), dtype=np.uint8)
    
    # Векторизация: создаем массив пикселей для классификации
    pixel_values = image.reshape(-1, 1)  # Преобразуем в 2D массив (n_samples, n_features)
    
    # Классификация всех пикселей за один раз
    classified_pixels = classifier.predict(pixel_values)
    
    # Заполняем классифицированное изображение
    classified_image = classified_pixels.reshape(height, width)
    
    print("Классификация всего изображения завершена.")
    return classified_image

classified_image = classify_image(raster_data, classifier)

# Шаг 10: Сохранение результата
output_path = 'classified_image.tif'
print(f"Сохранение результата в {output_path}...")

# Сохранение с параметрами геопривязки от исходного файла
with rasterio.open(raster_path) as src:
    # Получаем только параметры геопривязки
    meta = src.meta.copy()
    meta.update({
        'height': classified_image.shape[0],
        'width': classified_image.shape[1],
        'count': 1,  # Указываем количество каналов
        'dtype': 'uint8',  # Указываем тип данных
        'driver': 'GTiff'  # Указываем драйвер
    })
    
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(classified_image, 1)  # Записываем данные в первый канал

print("Сохранение завершено.")